In [1]:
import pyarrow.parquet as pq
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
table2 = pq.read_table('market_data.parquet')




In [3]:
table = table2.to_pandas()
table.shape

(1434082, 4)

In [4]:
# Flatten column names if they are multi-level
# Reset the index to include 'date' and 'id' as regular columns
table = table.reset_index()


In [5]:
table.head(10)

,date,id,price,mktcap,liquidity,sector
0,1999-05-06,1,461.0,1.235952e+09,129080.0,None
1,1999-05-07,1,455.0,1.219866e+09,4550.0,None
2,1999-05-10,1,455.0,1.219866e+09,910.0,None
3,1999-05-11,1,460.0,1.233271e+09,4600.0,None
4,1999-05-12,1,460.0,1.233271e+09,460.0,None
5,1999-05-14,1,460.0,1.233271e+09,4600.0,None
6,1999-05-19,1,460.0,1.233271e+09,2300.0,None
7,1999-05-21,1,460.0,1.233271e+09,4600.0,None
8,1999-05-26,1,460.0,1.233271e+09,13800.0,None
9,1999-05-31,1,463.0,1.241314e+09,13890.0,None


In [6]:
# Check the number of NA values in the entire DataFrame
na_count = table.isna().sum()

# Display the count of NA values for each column
na_count

date               0
id             43449
price              0
mktcap             0
liquidity     105235
sector       1431763
dtype: int64

In [7]:
# Save the DataFrame to a CSV file
# table.to_csv('market_data.csv', index=False)

In [8]:
# Check the columns in the DataFrame
print(table.columns)

Index(['date', 'id', 'price', 'mktcap', 'liquidity', 'sector'], dtype='object')


In [9]:
# Print the schema of the Parquet file to check for missing fields
print(table2.schema)

price: double
mktcap: double
liquidity: double
sector: string
date: timestamp[ns]
id: string
-- schema metadata --
pandas: '{"index_columns": ["date", "id"], "column_indexes": [{"name": nu' + 882


In [10]:
# Filter the DataFrame for rows where 'id' equals 1
df = table[table['id'] == '2']  # Ensure 'id' is treated as a string if necessary
df = df.drop(columns=['sector'])

# 1) rolling‐5 mean
roll5 = df['liquidity'].rolling(window=5, min_periods=1).mean()

# 2) fill NaNs with the rolling mean
df['liquidity'] = df['liquidity'].fillna(roll5)

# 3) for any still‐NaN at the top, backfill from the first real value
df['liquidity'].fillna(0, inplace=True)

# df.to_csv('test_1_imputed.csv', index=False)


C:\Users\IAB\AppData\Local\Temp\ipykernel_28228\3948143511.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['liquidity'].fillna(0, inplace=True)


In [11]:
df.shape

(7734, 5)

In [12]:
df.head(10)


,date,id,price,mktcap,liquidity
1863,1990-01-30,2,36.961449,765000000.0,0.0
1864,1990-01-31,2,36.526608,756000000.0,0.0
1865,1990-02-14,2,36.526608,756000000.0,0.0
1866,1990-02-15,2,36.526608,756000000.0,0.0
1867,1990-02-16,2,37.178869,769500000.0,0.0
1868,1990-02-22,2,36.961449,765000000.0,0.0
1869,1990-02-26,2,36.526608,756000000.0,0.0
1870,1990-02-27,2,36.961449,765000000.0,0.0
1871,1990-03-07,2,36.526608,756000000.0,0.0
1872,1990-03-12,2,36.091767,747000000.0,0.0


In [ ]:
import pandas as pd
import numpy as np

# Calculate 20-day EMA
df['ema_20'] = df['price'].ewm(span=20, adjust=False).mean()

# Calculate 50-day EMA
df['ema_50'] = df['price'].ewm(span=50, adjust=False).mean()

# Calculate 14-day rolling standard deviation (realized volatility)
df['realized_vol_14d'] = df['price'].rolling(window=14).std()

# Handle edge cases:
# For model training, you may want to:
# 1. Drop rows with NaN values
df_model = df.dropna()

# Or 2. Forward fill missing values (not recommended for the beginning of the series)
# df_model = df.fillna(method='ffill')

# Or 3. Drop specific columns where necessary
# First 13 days will have NaN for realized_vol_14d
# First 19 days will have less reliable ema_20
# First 49 days will have less reliable ema_50

# Create a boolean mask for rows we want to keep for the model
valid_data_mask = (~df['realized_vol_14d'].isna()) & (df.index >= 49)
df_model = df[valid_data_mask].copy()

# Drop the 'liquidity' column from df_model
df_model = df_model.drop(columns=['liquidity'])

# Print the first few rows of the original dataframe to see the NaN pattern
# print("Original DataFrame with new columns:")
# print(df.head(60))

# Print the first few rows of the filtered dataframe for modeling
print("\nFiltered DataFrame for modeling:")
print(df_model.head(10))

print(f"\nOriginal data: {len(df)} rows")
print(f"Modeling data: {len(df_model)} rows")


Filtered DataFrame for modeling:
           date id      price       mktcap     ema_20     ema_50  \
1876 1990-04-02  2  36.309151  780267180.0  36.581383  36.766945   
1877 1990-04-05  2  36.309151  780267180.0  36.555456  36.748992   
1878 1990-04-24  2  36.780699  790400520.0  36.576908  36.750236   
1879 1990-04-25  2  36.309151  780267180.0  36.551407  36.732938   
1880 1990-05-03  2  36.780699  790400520.0  36.573244  36.734811   
1881 1990-05-17  2  36.309151  780267180.0  36.548093  36.718119   
1882 1990-06-06  2  36.780699  790400520.0  36.570246  36.720573   
1883 1990-06-07  2  36.309151  780267180.0  36.545380  36.704439   
1884 1990-06-08  2  36.073378  775200510.0  36.500427  36.679691   
1885 1990-06-13  2  36.309151  780267180.0  36.482210  36.665160   

      realized_vol_14d  valid_for_model  
1876          0.366447             True  
1877          0.350138             True  
1878          0.358361             True  
1879          0.362447             True  
1880   

In [16]:
df_model.to_csv('test_v2.csv', index=False)